## 라이브러리 로딩

In [149]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

## 데이터 로딩 및 확인

In [150]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv')

In [151]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [152]:
train.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [153]:
test.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


## 결측치 확인 및 처리

In [154]:
train.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [155]:
test.isnull().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [156]:
# train.fillna(0,inplace = True)
# test.fillna(0,inplace = True)
# 중위수로 결측치 처리하기
train_na = train.fillna(train.median())
test_na = test.fillna(test.median())
test = test_na.drop(['id'],axis=1)
train_x = train_na.drop(['id','count'],axis = 1)
train_y = train_na['count']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=100)
x_train.shape, y_train.shape
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train_scaled = ss.transform(x_train)
test_scaled = ss.transform(test)

예측모델의 파이프라인 구축해서 각 모델의 점수를 비교해서 최상의 모델을 선택하고 하이퍼 파라메터적용하기

ploy증가하는 함수도 적용해서 선형이 아닌 비선형도 성능 측정하기

In [157]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso, SGDRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# 교차 검증
from sklearn.model_selection import cross_validate
# 특성을 추가하여 비 선형으로 만들기
from sklearn.preprocessing import PolynomialFeatures

# 1. 가능한 모든 모델로 측정한다.
temp1  = [
    'KNeighborsRegressor','LinearRegression','Ridge','Lasso', 'SGDRegressor','RandomForestRegressor','ExtraTreesRegressor',
    'DecisionTreeRegressor','XGBRegressor','LGBMRegressor'
]
temp2 = [
    KNeighborsRegressor(),LinearRegression(),Ridge(),Lasso(), SGDRegressor(),RandomForestRegressor(),ExtraTreesRegressor(),
    DecisionTreeRegressor(),XGBRegressor(),LGBMRegressor()
]
models = tuple( zip(temp1,temp2))
models

(('KNeighborsRegressor', KNeighborsRegressor()),
 ('LinearRegression', LinearRegression()),
 ('Ridge', Ridge()),
 ('Lasso', Lasso()),
 ('SGDRegressor', SGDRegressor()),
 ('RandomForestRegressor', RandomForestRegressor()),
 ('ExtraTreesRegressor', ExtraTreesRegressor()),
 ('DecisionTreeRegressor', DecisionTreeRegressor()),
 ('XGBRegressor',
  XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_est

In [158]:
import numpy as np
for name,model in models:
  score  = cross_validate(model,x_train_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.6693598286091967
LinearRegression : 0.5813469866793826
Ridge : 0.5813659358733126
Lasso : 0.5816589670822431
SGDRegressor : 0.5793593163322897
RandomForestRegressor : 0.7705387380944657
ExtraTreesRegressor : 0.7821067070653562
DecisionTreeRegressor : 0.545849408524066
XGBRegressor : 0.7609704731011966
LGBMRegressor : 0.758873200528528


In [159]:
# 비선형으로 예측하기 2차원
import numpy as np
pf = PolynomialFeatures(degree=2)
x_train_poly = pf.fit_transform(x_train)
ss = StandardScaler()
x_train_poly_scaled = ss.fit_transform(x_train_poly)
for name,model in models:
  score  = cross_validate(model,x_train_poly_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.657944573744866
LinearRegression : 0.6274844004727933
Ridge : 0.6312565126836854
Lasso : 0.6278180750181435
SGDRegressor : 0.6248966876563079
RandomForestRegressor : 0.7556536716197874
ExtraTreesRegressor : 0.790030020526246
DecisionTreeRegressor : 0.49696896400871127
XGBRegressor : 0.7473853054043834
LGBMRegressor : 0.7737105136641451


In [160]:
# 비선형으로 예측하기 3차원
import numpy as np
pf = PolynomialFeatures(degree=3)
x_train_poly = pf.fit_transform(x_train)
ss = StandardScaler()
x_train_poly_scaled = ss.fit_transform(x_train_poly)
for name,model in models:
  score  = cross_validate(model,x_train_poly_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.6660609700922837
LinearRegression : 0.5712343348279636
Ridge : 0.6690737877062002
Lasso : 0.6347010361655256
SGDRegressor : -3.494926218927749e+17
RandomForestRegressor : 0.7472281459552266
ExtraTreesRegressor : 0.7827903832878942
DecisionTreeRegressor : 0.4869309283506924
XGBRegressor : 0.762766466760462
LGBMRegressor : 0.7668470772970029


In [161]:
# 결정트리로 중요 피처만 골라서 재 학습
dt = DecisionTreeRegressor()
dt.fit(x_train,y_train)
np.sort(dt.feature_importances_), np.argsort(dt.feature_importances_)
feature_dict = dict(zip(x_train.columns, dt.feature_importances_))
sorted_items = sorted(feature_dict.items(), key=lambda x : x[1],reverse=True)
print(sorted_items)
top4cols = [a for a,b in sorted_items[:2]]
x_train_top4 = x_train.loc[:,top4cols]
# x_train_top4.head()
# 스케일링
from sklearn.preprocessing import RobustScaler
ss = RobustScaler()
x_train_top4_scaled = ss.fit_transform(x_train_top4)

for name,model in models:
  score  = cross_validate(model,x_train_top4_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

[('hour', 0.5888011640767341), ('hour_bef_temperature', 0.17619546955251073), ('hour_bef_ozone', 0.04696744729670609), ('hour_bef_humidity', 0.04483780532500811), ('hour_bef_windspeed', 0.03372000172370742), ('hour_bef_precipitation', 0.029363984356467185), ('hour_bef_visibility', 0.027154925964527826), ('hour_bef_pm2.5', 0.02710963051338448), ('hour_bef_pm10', 0.025849571190954077)]
KNeighborsRegressor : 0.6574307949617426
LinearRegression : 0.5412496511646111
Ridge : 0.5412582330993121
Lasso : 0.5408335273031938
SGDRegressor : 0.5410129368745313
RandomForestRegressor : 0.6676253604858349
ExtraTreesRegressor : 0.6046536741637212
DecisionTreeRegressor : 0.5483914386868975
XGBRegressor : 0.6364495127595005
LGBMRegressor : 0.6997137574661194


In [162]:
# 피처의 개수가 많아야 성능이 높아짐.... 
from sklearn.decomposition import PCA
ss = StandardScaler()
pf = PolynomialFeatures(degree=3)
pca = PCA(n_components=0.5)
x_train_pca = pca.fit_transform(x_train)
x_train_pca_scaled = ss.fit_transform(x_train_pca)
x_train_pca_scaled_poly = pf.fit_transform(x_train_pca_scaled)

for name,model in models:
  score  = cross_validate(model,x_train_pca_scaled_poly,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')



KNeighborsRegressor : -0.061406314322779744
LinearRegression : 0.07541234633768286
Ridge : 0.0754887937194777
Lasso : 0.07518388196740595
SGDRegressor : 0.07752500373397982
RandomForestRegressor : -0.29293579622580757
ExtraTreesRegressor : -0.46682174508913316
DecisionTreeRegressor : -0.7523027526579152
XGBRegressor : -0.21695443592993535
LGBMRegressor : 0.019402807973451463


In [163]:
# 가장 성능이 좋은 케이스
# 비선형으로 예측하기 2차원
import numpy as np
pf = PolynomialFeatures(degree=2)
pf.fit(x_train)
x_train_poly = pf.transform(x_train)
x_test_poly = pf.transform(x_test)
test_poly = pf.transform(test)

ss = StandardScaler()
ss.fit(x_train_poly)
x_train_poly_scaled = ss.transform(x_train_poly)
x_test_poly_scaled = ss.transform(x_test_poly)
test_poly_scaled = ss.transform(test_poly)

for name,model in models:
  score  = cross_validate(model,x_train_poly_scaled,y_train)
  print(f'{name} : {np.mean(score["test_score"])}')

KNeighborsRegressor : 0.657944573744866
LinearRegression : 0.6274844004727933
Ridge : 0.6312565126836854
Lasso : 0.6278180750181435
SGDRegressor : 0.6283604696552355
RandomForestRegressor : 0.7578683993916349
ExtraTreesRegressor : 0.7897057097361422
DecisionTreeRegressor : 0.5149845773866404
XGBRegressor : 0.7473853054043834
LGBMRegressor : 0.7737105136641451


In [164]:
# ExtraTreesRegressor 로 하이퍼 파라메터 수행하기
from sklearn.model_selection import RandomizedSearchCV
# 튜닝할 하이퍼파라미터 범위 설정
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]    
}
model = ExtraTreesRegressor()
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, random_state=42)
random_search.fit(x_train_poly_scaled, y_train)
# 최적의 하이퍼파라미터와 성능 출력
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Best Score: 0.7884718078423234


In [165]:
best_model = random_search.best_estimator_
best_model.score(x_test_poly_scaled,y_test) 

0.8160729707070813

## 학습 된 모델로 예측 데이터 생성

In [166]:
pred = best_model.predict(test_poly_scaled)
pred

array([102.415     , 244.99      ,  46.985     ,  59.675     ,
        50.83166667, 127.52666667, 171.47833333, 275.115     ,
        48.23666667, 137.68      , 288.71166667, 259.68      ,
       122.215     ,  35.79166667, 158.945     , 169.38166667,
        26.33166667, 148.14      , 340.31      , 149.725     ,
       209.07833333,  94.95166667,  35.415     , 119.85166667,
       135.34      , 114.68833333,  24.04166667, 103.17666667,
       102.545     , 176.50166667,  63.99333333,  34.07333333,
        66.76166667, 134.525     , 248.5       ,  65.27166667,
       135.13666667,  85.91166667, 250.01166667,  93.95666667,
        50.76833333, 118.56833333, 165.52166667, 118.365     ,
       320.14166667, 215.995     ,  87.58166667,  65.03666667,
        21.22833333,  87.275     , 216.1       ,  91.65333333,
       140.185     , 106.63      , 190.83166667, 121.77666667,
        45.78      , 173.90666667,  26.82666667,  18.25166667,
        94.86666667,  89.38333333, 230.55      , 296.90

## 제출파일 생성

In [174]:
submission = pd.read_csv('submission.csv')

In [175]:
submission['count'] = pred

In [177]:
submission.head()

,id,count
0,0,102.415000
1,1,244.990000
2,2,46.985000
3,4,59.675000
4,5,50.831667


In [178]:
submission.to_csv('베이스라인.csv',index = False)